# Simple Script to Test Reducing the Number of Faces in Shapes of ShapeNetCore

You can download the dataset here: https://huggingface.co/datasets/ShapeNet/shapenetcore-glb

You have to generate an account on huggingface and apply for access to the dataset. Once you have access, you can download the dataset. See here for more information on using the Hugginface CLI (Command Line Interface) and downloading huggingface repositories : https://huggingface.co/docs/huggingface_hub/main/en/guides/cli

In [1]:
import trimesh

In [2]:
# load a specific mesh file from the shapenet dataset - Here it is a gltf file but glb and obj are also fine... 
gltf_path = '/mnt/damian/Projects/meshgpt/shapenet/shapenetcore-gltf/car/1a0bc9ab92c915167ae33d942430658c.gltf'

# Load the mesh
mesh = trimesh.load(gltf_path, force='mesh')

# Check if the mesh is loaded correctly
if not mesh.is_empty:
    print("Mesh loaded successfully!")
else:
    print("Failed to load the mesh.")

# Scale the mesh to fit within a unit cube
mesh.apply_scale(1 / mesh.scale)

# Translate the mesh to the origin
mesh.apply_translation(-mesh.centroid)

Mesh loaded successfully!


In [ ]:
# Visualize the mesh
mesh.show()

In [4]:
import numpy as np

# Convert the mesh vertices and faces to a numpy array
vertices = np.array(mesh.vertices)
faces = np.array(mesh.faces)

## Reduce Faces with Pymeshlab 
pymeshlab is a library that allows to perform several operations on 3D models. In this notebook, we will use it to reduce the number of faces in the shapes of ShapeNetCore.

In [117]:
import pymeshlab

Load the mesh into a pymeshlab object:

In [188]:
# create mesh based on the vertices and faces
ml_mesh = pymeshlab.Mesh(vertex_matrix=vertices, face_matrix=faces)

# create a new MeshSet
ms = pymeshlab.MeshSet()

# add the mesh to the MeshSet
# ms.add_mesh(ml_mesh, "cube_mesh")
ms.add_mesh(ml_mesh)

Decimate (reduce) the number of faces:

In [191]:
# thresholdAbsoluteValue
ms.meshing_merge_close_vertices(threshold=pymeshlab.PureValue(0.045))

# ms.meshing_decimation_clustering(
#     threshold=pymeshlab.PercentageValue(5))
ms.meshing_decimation_quadric_edge_collapse(
    targetfacenum=1000, preservenormal=True, planarquadric=True, preserveboundary=True, preservetopology=True)  # 

# print the number of faces
print("Number of faces after decimation: ", ms.current_mesh().face_number())

Number of faces after decimation:  999


Save the mesh with the reduced number of faces:

In [192]:
ms.save_current_mesh('./outputs/simplified_model.obj')

Visualize the reduced mesh:

In [193]:
# load the simplified mesh
simple_mesh = trimesh.load('./outputs/simplified_model.obj', force='mesh')

# Visualize the mesh
simple_mesh.show()

## Processing Multiple Shapes - Reduce Faces on Several Shapes

First load all cars in the ShapeNetCore dataset:

In [194]:
# get all gltf files in /mnt/damian/Projects/meshgpt/shapenet/shapenetcore-gltf/car/
import os

gltf_dir = '/mnt/damian/Projects/meshgpt/shapenet/shapenetcore-gltf/car/'

gltf_files = [os.path.join(gltf_dir, f) for f in os.listdir(gltf_dir) if f.endswith('.gltf')]

print("Number of gltf files: ", len(gltf_files))

Number of gltf files:  3514


Write a function to reduce the number of faces in a shape:

In [239]:
import numpy as np
import pymeshlab
import copy

def load_gltf_and_decimate_mesh(file_path):
    # Load the mesh
    mesh = trimesh.load(file_path, force='mesh')

    # Check if the mesh is loaded correctly
    if not mesh.is_empty:
        print("Mesh loaded successfully!")
    else:
        print("Failed to load the mesh.")

    # Scale the mesh to fit within a unit cube
    mesh.apply_scale(1 / mesh.scale)

    # Translate the mesh to the origin
    mesh.apply_translation(-mesh.centroid)

    vertices = np.array(mesh.vertices)
    faces = np.array(mesh.faces)

    # create mesh based on the vertices and faces
    mesh = pymeshlab.Mesh(vertex_matrix=vertices, face_matrix=faces)

    # print number of faces
    print("Number of faces before decimation: ", mesh.face_number())

    # go through a decimation_range of 0.001 to 0.1 until we reach the target number of faces
    n_target_faces = 1000
    decimation_range = np.linspace(0.001, 0.1, 1000)

    for threshold in decimation_range:
        # thresholdAbsoluteValue
        ml_mesh = pymeshlab.Mesh(
            vertex_matrix=vertices.copy(), face_matrix=faces.copy())

        # create a new MeshSet
        ms = pymeshlab.MeshSet()

        # add the mesh to the MeshSet
        ms.add_mesh(ml_mesh)
        ms.meshing_merge_close_vertices(
            threshold=pymeshlab.PureValue(threshold))
        
        if ms.current_mesh().face_number() <= n_target_faces:
            break

    # # thresholdAbsoluteValue
    # ms.meshing_merge_close_vertices(threshold=pymeshlab.PureValue(0.045))

    # ms.meshing_decimation_quadric_edge_collapse(
    #     targetfacenum=800, preservenormal=True, planarquadric=True, preserveboundary=True, preservetopology=True)  # 

    # print the number of faces
    print("Number of faces after decimation: ", ms.current_mesh().face_number())

    output_path = file_path.replace('shapenetcore-gltf', 'shapenetcore-gltf-simplified')
    output_path = output_path.replace('.gltf', '_simplified.obj')
    ms.save_current_mesh(output_path)

    return output_path

Let's test the function on a single shape:

In [240]:
object_path = load_gltf_and_decimate_mesh(gltf_files[5])

# load the simplified mesh
simple_mesh = trimesh.load(object_path, force='mesh')

# Visualize the mesh
simple_mesh.show()

Mesh loaded successfully!
Number of faces before decimation:  81459
Number of faces after decimation:  997


Let's test the function on more shapes:

In [241]:
number_of_files = 30
object_paths = []
for i in range(number_of_files):
    object_path = load_gltf_and_decimate_mesh(gltf_files[i])
    print("Processed file: ", object_path)
    object_paths.append(object_path)

Mesh loaded successfully!
Number of faces before decimation:  51500
Number of faces after decimation:  997
Processed file:  /mnt/damian/Projects/meshgpt/shapenet/shapenetcore-gltf-simplified/car/f2f62db1a9e5823a7aaf6e2fd8453e07_simplified.obj
Mesh loaded successfully!
Number of faces before decimation:  116165
Number of faces after decimation:  981
Processed file:  /mnt/damian/Projects/meshgpt/shapenet/shapenetcore-gltf-simplified/car/80bdcb908765eb9ec349f2dfac43a4cf_simplified.obj
Mesh loaded successfully!
Number of faces before decimation:  124333
Number of faces after decimation:  987
Processed file:  /mnt/damian/Projects/meshgpt/shapenet/shapenetcore-gltf-simplified/car/c44111ca55c697117fbfaeeb4819ffd2_simplified.obj
Mesh loaded successfully!
Number of faces before decimation:  61326
Number of faces after decimation:  989
Processed file:  /mnt/damian/Projects/meshgpt/shapenet/shapenetcore-gltf-simplified/car/ac3585c59a80b822b4d8eb72d337b2e3_simplified.obj
Mesh loaded successfully!


Ok, let's generate a scene with the all reduced shapes:

In [251]:
# go through all the files and generate scene of all the simplified meshes by loading them and adding them to the scene and shifting them
import trimesh

scene = trimesh.Scene()

for i, object_path in enumerate(object_paths):
    mesh = trimesh.load(object_path, force='mesh')
    mesh.apply_translation([i, 0, 0])
    scene.add_geometry(mesh)

# safe the scene as obj
scene.export('./outputs/scene.obj')

'# https://github.com/mikedh/trimesh\n\no f2f62db1a9e5823a7aaf6e2fd8453e07_simplified.obj\nv -0.04063327 0.00046470 -0.01855863 1.00000000 1.00000000 1.00000000\nv -0.07481013 0.05142946 0.02671149 1.00000000 1.00000000 1.00000000\nv -0.03400452 0.09976892 0.03877185 1.00000000 1.00000000 1.00000000\nv -0.11561674 0.09976892 0.03877185 1.00000000 1.00000000 1.00000000\nv -0.10898599 0.00046470 -0.01855863 1.00000000 1.00000000 1.00000000\nv -0.07481013 -0.01127262 -0.06032334 1.00000000 1.00000000 1.00000000\nv -0.12892824 -0.00411081 -0.09898772 1.00000000 1.00000000 1.00000000\nv -0.02069102 -0.00411081 -0.09898772 1.00000000 1.00000000 1.00000000\nv -0.07481013 -0.01414295 0.02229399 1.00000000 1.00000000 1.00000000\nv 0.03565335 0.03749889 0.00107059 1.00000000 1.00000000 1.00000000\nv 0.07273253 0.10074203 0.03066693 1.00000000 1.00000000 1.00000000\nv 0.10981173 0.03749889 0.00107059 1.00000000 1.00000000 1.00000000\nv 0.12870186 -0.00234161 -0.04528665 1.00000000 1.00000000 1.00

Let's visualize them: 

In [252]:
scene.show()